In [1]:
from datetime import datetime
import subprocess as sp
import xarray as xr
import os


# CMEMS credentials required to download the data
username = 'your_username'
password = 'your_password'

#Variables to download
variables = ['analysed_sst', 'analysis_error']
var=' '.join(['--variable '+u for u in variables])

#Creating output Folder
ofo = '../../data/sst_l4/'
if not os.path.exists(ofo):  
    os.makedirs(ofo)

#Boundaries of the Mediterranean Sea
min_lon = "-5.61"
max_lon = "36.32"
min_lat = "30.12"
max_lat = "46.02"
    
year_start = 1982
year_end = 2021
years = list(range(year_start,year_end+1))

In [ ]:
for year in years:
    yyyystart = year
    mmstart = 1
    ddstart = 1
    hhstart = " 00:00:00"
    yyyyend = year
    mmend = 12
    ddend = 31
    hhend = " 00:00:00"

    # Modifying the '20xx-x-x' type of string into '20xx-0x-0x'.

    pre_datestart = "{0}-{1}-{2} {3}".format(yyyystart, mmstart, ddstart, hhstart)
    dt = datetime.strptime(pre_datestart, "%Y-%m-%d %H:%M:%S")
    datestart = datetime.strftime(dt, "%Y-%m-%d %H:%M:%S")

    pre_dateend = "{0}-{1}-{2} {3}".format(yyyyend, mmend, ddend, hhend)
    dt = datetime.strptime(pre_dateend, "%Y-%m-%d %H:%M:%S")
    dateend = datetime.strftime(dt, "%Y-%m-%d %H:%M:%S")

    # Output filename
    ofi = "SST_L4_{}.nc".format(year)

    script = """python -m motuclient --motu https://my.cmems-du.eu/motu-web/Motu --service-id SST_MED_SST_L4_REP_OBSERVATIONS_010_021-TDS --product-id cmems_SST_MED_SST_L4_REP_OBSERVATIONS_010_021 --longitude-min {0} --longitude-max {1} --latitude-min {2} --latitude-max {3} --date-min "{4}" --date-max "{5}" {6} --out-dir {7} --out-name {8} --user {9} --pwd {10}
    """.format(
        min_lon,
        max_lon,
        min_lat,
        max_lat,
        datestart,
        dateend,
        var,
        ofo,
        ofi,
        username,
        password,
    )

    output = sp.getoutput(script)
    print("Year {} downloaded".format(year))

    # Removing the remaining patch of atlantic ocean at Cantabrian Sea

    lat_0 = 42.843059
    lon_0 = -0.179951

    file = ofo + ofi
    ds = xr.open_dataset(file)
    ds = ds.where((ds.lat < lat_0) | (ds.lon > lon_0))

    os.remove(file)

    ds.to_netcdf(file)
    print("Year {} cropped".format(year))